<a href="https://colab.research.google.com/github/momo0510/project01/blob/main/%E3%80%8CFinalProj_Example_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境設定

進入程式請先執行下面的 `! pip install Backtesting`

In [ ]:
! pip install Backtesting
! pip install pandas-ta
!pip install pandas
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=3630e48b39e66c59e4c849bd684edaf204a113a095cc428398b010c1b554c8c8
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built Backtesting
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=17802b0462f3f16e6c4577b194ed6f9330c1d433b5b1cc5c7db1d14381330485
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
1

# 修改套件問題


修改`/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py`的148行

改成
`new_bar_idx = new_index.get_indexer([mean_time], method='nearest')[0]`

In [ ]:
import fileinput
import sys

# 文件路徑
file_path = '/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py'

# 需要修改的行號
line_to_edit = 148

# 新的內容
new_content = "                new_bar_idx = new_index.get_indexer([mean_time], method='nearest')[0]"

# 安全檢查：備份原始文件
backup_file_path = file_path + '.backup'
with open(file_path, 'r') as original_file, open(backup_file_path, 'w') as backup_file:
    backup_file.write(original_file.read())

# 嘗試修改文件
try:
    with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
        for i, line in enumerate(file, start=1):
            if i == line_to_edit:
                print(new_content)  # 替換為新內容
            else:
                print(line, end='')  # 保留原始內容

    print("文件修改成功")
except Exception as e:
    print(f"發生錯誤: {e}", file=sys.stderr)
    # 發生錯誤時恢復原始文件
    with open(backup_file_path, 'r') as backup_file, open(file_path, 'w') as original_file:
        original_file.write(backup_file.read())
    print("文件已恢復到原始狀態。", file=sys.stderr)


文件修改成功


# 主程式

In [ ]:
# 掛載自己的雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.test import SMA, GOOG
import pandas as pd
import pandas_ta as ta
# 請更換成自己檔案在雲端上的路徑
data_path = "/content/drive/MyDrive/EURUSD_M1_train.csv"

#匯入資料
df = pd.read_csv(data_path, names=GOOG.columns)
df.index = pd.to_datetime(df.index)
df.drop(index=df.index[0], inplace=True)
df = df.astype("float")
df = df[:50000].copy()
df.head()

<ipython-input-9-ef8cd9f8cb4a>:9: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, names=GOOG.columns)


,Open,High,Low,Close,Volume
2017-01-02 02:00:00,1.05155,1.05197,1.05155,1.05190,0.0
2017-01-02 02:01:00,1.05209,1.05209,1.05177,1.05179,0.0
2017-01-02 02:02:00,1.05177,1.05198,1.05177,1.05178,0.0
2017-01-02 02:03:00,1.05188,1.05200,1.05188,1.05200,0.0
2017-01-02 02:04:00,1.05196,1.05204,1.05196,1.05203,0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 新增區段

In [ ]:
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI']=ta.rsi(df.Close, length=25)
my_bbands = ta.bbands(df.Close, length=15, std=2.5)
df=df.join(my_bbands)

VWAPsignal = [0]*len(df)
backcandles = 20

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i],df.Close[i])>=df.VWAP[i]:
            dnt=0
        if min(df.Open[i],df.Close[i])<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal


TypeError: ignored

In [ ]:
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_15_2.5'][l]
        and df.RSI[l]<45 and df.RSI[l]>20):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_15_2.5'][l]
        and df.RSI[l]>55 and df.RSI[l]<80):
            return 1
    return 0

TotSignal = [0]*len(df)
for row in range(backcandles, len(df)):
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

In [ ]:
df[df.TotalSignal!=0].count()

In [ ]:
df[df.TotalSignal!=0].count()
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
def SIGNAL():
    return df.TotalSignal

In [ ]:
class mystrategy(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSL_Ratio = 1.5

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSL_Ratio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSL_Ratio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)



## 執行回測

### 設定
* 初始金額：100000000
* 手續費：0.00005

In [ ]:
bt = Backtest(df, mystrategy,cash=100000000, commission = 0.00005, exclusive_orders=True)

output = bt.run()

output

Start                     2019-09-30 00:00:00
End                       2022-09-30 20:55:00
Duration                   1096 days 20:55:00
Exposure Time [%]                    0.981381
Equity Final [$]             102176405.808351
Equity Peak [$]              102290870.735323
Return [%]                           2.176406
Buy & Hold Return [%]              -10.401623
Return (Ann.) [%]                    0.578872
Volatility (Ann.) [%]                 0.64252
Sharpe Ratio                         0.900941
Sortino Ratio                        1.597456
Calmar Ratio                         0.746946
Max. Drawdown [%]                   -0.774986
Avg. Drawdown [%]                   -0.069453
Max. Drawdown Duration      342 days 00:05:00
Avg. Drawdown Duration       13 days 00:55:00
# Trades                                  313
Win Rate [%]                        53.354633
Best Trade [%]                       0.415099
Worst Trade [%]                     -0.278205
Avg. Trade [%]                    

### 繪製分析圖

In [ ]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '10T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py

GridPlot(id='p1199', ...)